In [1]:
import carla
import math
import random
import numpy as np
import cv2
import subprocess
import time
import matplotlib

In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [3]:
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [4]:
vehicle_bp = bp_lib.find('vehicle.dodge.charger')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [5]:
# Start the traffic light controller in the background
traffic_light_process = subprocess.Popen(['python', 'traffic_light_controller.py'])
print("Traffic light control is running in the background!")

# Start the traffic script in the background
traffic_process = subprocess.Popen(['python', 'generate_traffic.py', '--number-of-vehicles', '40', '--number-of-walkers', '10'])

print("Traffic generation is running in the background!")

time.sleep(5)

# Set up traffic manager
traffic_manager = client.get_trafficmanager()
vehicle.set_autopilot(True, traffic_manager.get_port())

Traffic light control is running in the background!
Traffic generation is running in the background!


In [13]:
cameras = subprocess.Popen(['python', 'camera_processing.py'])
print("Cameras and Sensors working in background!")

Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Cameras and Sensors working in background!
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or

In [8]:
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(x=0.0, y=0.0, z=1.6), carla.Rotation(pitch=0.0))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

gnss_bp = bp_lib.find('sensor.other.gnss')
gnss_sensor = world.spawn_actor(gnss_bp, carla.Transform(), attach_to=vehicle)

imu_bp = bp_lib.find('sensor.other.imu')
imu_sensor = world.spawn_actor(imu_bp, carla.Transform(), attach_to=vehicle)

collision_bp = bp_lib.find('sensor.other.collision')
collision_sensor = world.spawn_actor(collision_bp, carla.Transform(), attach_to=vehicle)

lane_inv_bp = bp_lib.find('sensor.other.lane_invasion')
lane_inv_sensor = world.spawn_actor(lane_inv_bp, carla.Transform(), attach_to=vehicle)

obstacle_bp = bp_lib.find('sensor.other.obstacle')
obstacle_bp.set_attribute('hit_radius', '2')# 0.5
obstacle_bp.set_attribute('distance', '50')

# obstacle_sensor = world.spawn_actor(obstacle_bp, carla.Transform(), attach_to=vehicle)

obstacle_sensor = world.spawn_actor(
    obstacle_bp,
    carla.Transform(carla.Location(x=2.5, z=1.5)),  # Move sensor forward & slightly up
    attach_to=vehicle
)


In [9]:
# Callback function for the RGB camera sensor
def rgb_callback(image, data_dict):
    data = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data = cv2.cvtColor(data, cv2.COLOR_BGRA2BGR)  # Convert BGRA to BGR
    data_dict['rgb_image'] = data


# Callback function for the GNSS sensor
def gnss_callback(data, data_dict):
    # Stores latitude and longitude in the data dictionary
    data_dict['gnss'] = [data.latitude, data.longitude]

# Callback function for the IMU (Inertial Measurement Unit) sensor
def imu_callback(data, data_dict):
    # Stores gyroscope, accelerometer, and compass data in a structured format
    data_dict['imu'] = {
        'gyro': data.gyroscope,
        'accel': data.accelerometer,
        'compass': data.compass
    }

# Callback function for detecting lane invasions
def lane_inv_callback(event, data_dict):
    data_dict['lane_invasion'] = True

# Callback function for detecting collisions
def collision_callback(event, data_dict):
    data_dict['collision'] = True

# def obstacle_callback(event, data_dict, camera, k_mat, image_w, image_h):
#     # Ensure only vehicle and pedestrian obstacles are considered
#     if not ('vehicle' in event.other_actor.type_id or 'walker' in event.other_actor.type_id):
#         return  # Ignore other types of obstacles

#     # Append obstacle information to the data dictionary
#     data_dict.setdefault('obstacle', []).append({
#         'transform': event.other_actor.type_id,
#         'frame': event.frame
#     })

#     # Convert world coordinates to camera coordinates
#     world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

#     # Get the world location of the detected obstacle
#     obstacle_location = event.other_actor.get_location()

#     # Compute the image coordinates of the detected obstacle
#     image_point = get_image_point(obstacle_location, k_mat, world_2_camera)

#     # Ensure 'rgb_image' exists in data_dict
#     if 'rgb_image' not in data_dict:
#         print("Warning: 'rgb_image' not found in data_dict.")
#         return

#     # Draw a red circle on the RGB image at the detected obstacle's location
#     if 0 < image_point[0] < image_w and 0 < image_point[1] < image_h:
#         cv2.circle(data_dict['rgb_image'], tuple(image_point), 10, (0, 0, 255), 2)

def obstacle_callback(event, data_dict, camera, k_mat, image_w, image_h):
    print(f"Obstacle detected: {event.other_actor.type_id}")  # Debugging output

    if not ('vehicle' in event.other_actor.type_id or 'walker' in event.other_actor.type_id):
        print("Not a vehicle or pedestrian, ignoring.")
        return  # Ignore other types of obstacles

    data_dict.setdefault('obstacle', []).append({
        'transform': event.other_actor.type_id,
        'frame': event.frame
    })

    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
    obstacle_location = event.other_actor.get_location()

    image_point = get_image_point(obstacle_location, k_mat, world_2_camera)
    print(f"Projected 2D Point: {image_point}")  # Debugging output

    if 'rgb_image' not in data_dict or data_dict['rgb_image'] is None:
        print("Warning: RGB Image is missing!")
        return

    if 0 < image_point[0] < image_w and 0 < image_point[1] < image_h:
        print("Drawing obstacle on image...")
        cv2.circle(data_dict['rgb_image'], tuple(image_point), 10, (0, 0, 255), 2)  # Red circle


In [10]:
def build_projection_matrix(w, h, fov):
    """
    Constructs a 3x3 camera intrinsic matrix using width, height, and field of view (FOV).

    Parameters:
    w (float): Image width.
    h (float): Image height.
    fov (float): Field of view in degrees.

    Returns:
    np.ndarray: A 3x3 camera intrinsic matrix.
    """
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))  # Compute focal length
    K = np.identity(3)  # Initialize identity matrix

    K[0, 0] = K[1, 1] = focal  # Set focal length for x and y axes
    K[0, 2] = w / 2.0  # Set principal point in x direction
    K[1, 2] = h / 2.0  # Set principal point in y direction

    return K  # Return the intrinsic matrix


def get_image_point(loc, K, w2c):
    """
    Calculates the 2D projection of a 3D coordinate using a camera projection matrix.

    Parameters:
    loc (carla.Position): The 3D location of the object.
    K (np.ndarray): The camera intrinsic matrix.
    w2c (np.ndarray): The world-to-camera transformation matrix.

    Returns:
    tuple: The 2D pixel coordinates (x, y).
    """

    # Format the input coordinate (homogeneous representation)
    point = np.array([loc.x, loc.y, loc.z, 1])

    # Transform the point to camera coordinates
    point_camera = np.dot(w2c, point)

    # Convert from Unreal Engine 4 (UE4) coordinate system to standard:
    # UE4 uses (x, y, z), whereas the standard is (y, -z, x)
    # Also, remove the fourth component
    point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

    # Project 3D point to 2D using the camera intrinsic matrix
    point_img = np.dot(K, point_camera)

    # Normalize the projected image coordinates
    point_img[0] /= point_img[2]
    point_img[1] /= point_img[2]

    # Return as integer pixel coordinates
    return tuple(map(int, point_img[0:2]))

# Compute the world-to-camera transformation matrix
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()  # Camera image width
image_h = camera_bp.get_attribute("image_size_y").as_int()  # Camera image height
fov = camera_bp.get_attribute("fov").as_float()  # Camera field of view

# Compute the camera intrinsic matrix using the retrieved attributes
K = build_projection_matrix(image_w, image_h, fov)

In [11]:
# Retrieve camera image dimensions from the blueprint attributes
image_w = camera_bp.get_attribute("image_size_x").as_int()  # Camera image width
image_h = camera_bp.get_attribute("image_size_y").as_int()  # Camera image height

# Counters to track collisions and lane invasions
collision_counter = 20
lane_invasion_counter = 20

# Initialize sensor data dictionary to store various sensor readings
sensor_data = {
    'rgb_image': np.zeros((image_h, image_w, 4)),  # Initialize an empty RGBA image
    'collision': False,  # Flag for collision detection
    'lane_invasion': False,  # Flag for lane invasion detection
    'gnss': [0, 0],  # GPS coordinates (latitude, longitude)
    'obstacle': [],  # List to store detected obstacles
    'imu': {  # IMU (Inertial Measurement Unit) sensor data
        'gyro': carla.Vector3D(),  # Gyroscope readings
        'accel': carla.Vector3D(),  # Accelerometer readings
        'compass': 0  # Compass heading
    }
}

# Set up sensor listeners to capture data from CARLA sensors in real time
camera.listen(lambda image: rgb_callback(image, sensor_data))  
collision_sensor.listen(lambda event: collision_callback(event, sensor_data))  
gnss_sensor.listen(lambda event: gnss_callback(event, sensor_data))  
imu_sensor.listen(lambda event: imu_callback(event, sensor_data))  
lane_inv_sensor.listen(lambda event: lane_inv_callback(event, sensor_data))  
obstacle_sensor.listen(lambda event: obstacle_callback(event, sensor_data, camera, K, image_w, image_h))

In [12]:
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
fontColor = (255, 255, 255)
thickness = 2
lineType = 2

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', sensor_data['rgb_image'])
cv2.waitKey(1)


def draw_compass(img, theta):
    compass_center = (700, 100)
    compass_size = 50
    cardinal_directions = [
        ('N', [0, -1]),
        ('E', [1, 0]),
        ('S', [0, 1]),
        ('W', [-1, 0])
    ]
    for label, offset in cardinal_directions:
        cv2.putText(
            img, label,
            (int(compass_center[0] + 1.2 * compass_size * offset[0]), 
             int(compass_center[1] + 1.2 * compass_size * offset[1])),
            font, fontScale, fontColor, thickness, lineType
        )
    compass_point = (
        int(compass_center[0] + compass_size * math.sin(theta)),
        int(compass_center[1] - compass_size * math.cos(theta))
    )
    cv2.line(img, compass_center, compass_point, (255, 255, 255), 3)

# Initialize counters for collision and lane invasion events
collision_counter = 20
lane_invasion_counter = 20

while True:
    # Overlay GNSS data
    cv2.putText(sensor_data['rgb_image'], 
                'Lat: ' + str(sensor_data['gnss'][0]),
                (10, 30), font, fontScale, fontColor, thickness, lineType)
    cv2.putText(sensor_data['rgb_image'], 
                'Long: ' + str(sensor_data['gnss'][1]),
                (10, 50), font, fontScale, fontColor, thickness, lineType)
    
    # Compute adjusted acceleration (subtract gravity)
    accel = sensor_data['imu']['accel'] - carla.Vector3D(x=0, y=0, z=9.81)
    cv2.putText(sensor_data['rgb_image'], 
                'Accel: ' + str(accel.length()),
                (10, 70), font, fontScale, fontColor, thickness, lineType)
    
    # Gyroscope data
    cv2.putText(sensor_data['rgb_image'], 
                'Gyro: ' + str(sensor_data['imu']['gyro'].length()),
                (10, 100), font, fontScale, fontColor, thickness, lineType)
    
    # Compass heading
    cv2.putText(sensor_data['rgb_image'], 
                'Compass: ' + str(sensor_data['imu']['compass']),
                (10, 120), font, fontScale, fontColor, thickness, lineType)
    
    # Draw the compass visualization
    draw_compass(sensor_data['rgb_image'], sensor_data['imu']['compass'])
    
    # Handle collision events
    if sensor_data['collision']:
        collision_counter -= 1
        if collision_counter <= 1:
            sensor_data['collision'] = False
        cv2.putText(sensor_data['rgb_image'], 
                    'COLLISION',  
                    (250, 300), cv2.FONT_HERSHEY_SIMPLEX, 2,
                    (255, 255, 255), 3, 2)
    else:
        collision_counter = 20

    # Handle lane invasion events
    if sensor_data['lane_invasion']:
        lane_invasion_counter -= 1
        if lane_invasion_counter <= 1:
            sensor_data['lane_invasion'] = False
        cv2.putText(sensor_data['rgb_image'], 
                    'LANE INVASION',  
                    (190, 350), cv2.FONT_HERSHEY_SIMPLEX, 2,
                    (255, 255, 255), 3, 2)
    else:
        lane_invasion_counter = 20

    # Update the OpenCV window with the latest image
    cv2.imshow('RGB Camera', sensor_data['rgb_image'])
    
    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Cleanup operations
cv2.destroyAllWindows()

Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: static.road
Not a vehicle or pedestrian, ignoring.
Obstacle detected: s